In [7]:
import pandas as pd
import os
import time

def ensure_file_available(file_path):
    if not os.path.exists(file_path):
        print("File is not available. Please check your file in OneDrive")
        return False
    print("File is available. Proceeding with the processes")
    return True

def convert_numeric_to_datetime(df, column_name):
    for col in column_name:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[col] = pd.to_datetime(df[col], origin = '1899-12-30', unit = 'D', errors='coerce')


def clean_PO_HBA_file():
    # Define source and save path
    source_file = r'C:\Users\Thanawit C\OneDrive - Sahamit Product Co.,Ltd\Data for Stock Report\COPY_PO HBA.xlsb'
    save_path = r'D:\Data for Stock Report\cleaned_PO_pending_HBA.xlsx'

    ensure_file_available(source_file)

    try:
        # Load the Excel file with specified sheet name and header row
        df = pd.read_excel(source_file, sheet_name='PendingPO', header=0)

        # Trim extra space in header names
        df.columns = df.columns.str.strip()

        # Select the needed columns
        selected_columns = df[['CJ_Article', 
                               'SHM_Article',
                               'SHM_PO NO.',
                               'CJ_PO Date',
                               'CJ_PO NO.',
                               'CJ_Description',
                               'DC_Location',
                               'PO Pending',
                               'PO_Status',
                               'Next Delivery',
                               'Supplier_Short_Name']]
        
        # Rename columns
        selected_columns.rename(columns={
            'CJ_PO Date': 'SHM_PO_Date',
            }, inplace=True)

        # Filter the DataFrame with specific conditions
        selected_columns = selected_columns[selected_columns['PO_Status'].str.strip().str.lower() == 'pending']
        selected_columns = selected_columns[selected_columns['CJ_Article'].notna() & (selected_columns['CJ_Article'] != 0)]

        # cast data to a str
        print(selected_columns.dtypes)
        selected_columns['CJ_Article'] = selected_columns['CJ_Article'].astype(str)
        selected_columns['CJ_Article'] = selected_columns['CJ_Article'].apply(lambda x: x.rstrip('0').rstrip('.')) 
        selected_columns['SHM_Article'] = selected_columns['SHM_Article'].astype(str)

        # cast a date column to be numeric with function
        convert_numeric_to_datetime(selected_columns, ['SHM_PO_Date', 'Next Delivery'])

        # Add new columns using .loc
        selected_columns.loc[:, 'DC'] = selected_columns['DC_Location'].map({
            'D001': 'DC1',
            'D002': 'DC2',
            'D004': 'DC4',
            'TD09': 'TD09'
        })
        # Read other sheet in the same file to merge with selected_columns on 'SHM_Article'
        df2 = pd.read_excel(source_file, sheet_name='SupplyRecord', header=0)
        selected_df2 = df2[['SHM_Article', 'Unit_PC/CAR']]
        selected_df2.rename(columns={'Unit_PC/CAR': 'PC_Cartons'}, inplace=True)
        selected_df2['SHM_Article'] = selected_df2['SHM_Article'].astype(str)
        # Merge the two DataFrames on 'SHM_Article'
        selected_columns = pd.merge(selected_columns, selected_df2, on='SHM_Article', how='left')
       
        # Create New column for pending PO in Cartons
        selected_columns.loc[:, 'PendingPO (CTN)'] = selected_columns['PO Pending'] / selected_columns['PC_Cartons']
        selected_columns.loc[:, 'PendingPO (CTN)'] = selected_columns['PendingPO (CTN)'].round(0).astype(int)

        # Pivot data
        pivoted_df = selected_columns.pivot_table(
            index=['CJ_Article', 'SHM_Article'],
            columns=['DC'],
            values='PO Pending',
            aggfunc='sum',
            fill_value=0
        ).reset_index()

        pivoted_df.columns = ['CJ_Item', 'SHM_Item'] + [f'PO_Qty_to_{col}' for col in pivoted_df.columns[2:]]

        pivoted_min_del_date = selected_columns.pivot_table(
            index=['CJ_Article', 'SHM_Article'],
            columns='DC',
            values='Next Delivery',
            aggfunc='min'
        ).reset_index()

        # Rename column in pivot
        pivoted_min_del_date.columns = ['CJ_Item', 'SHM_Item'] + [f'Min_del_date_to_{col}' for col in pivoted_min_del_date.columns[2:]]

        # Merge 2 pivot tables for Min Del date
        merged_df = pd.merge(pivoted_df, pivoted_min_del_date, on=['CJ_Item', 'SHM_Item'], how='left')

        print(f"Checking data type of each column: {merged_df.dtypes}")

        # Save the cleaned data as an .xlsx file
        with pd.ExcelWriter(save_path, mode='w') as writer:
            merged_df.to_excel(writer, sheet_name='Pivot HBA', index=False)
            selected_columns.to_excel(writer, sheet_name='cleaned data', index=False)
        print(f"PO pending HBA report has been saved to {save_path}")

    except Exception as e:
        print(f"Error processing the Excel file: {e}")

# Call the function if the script is run directly
if __name__ == "__main__":
    clean_PO_HBA_file()


File is available. Proceeding with the processes


C:\Users\Thanawit C\AppData\Local\Temp\ipykernel_3060\2140401721.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns.rename(columns={


CJ_Article             float64
SHM_Article             object
SHM_PO NO.              object
SHM_PO_Date            float64
CJ_PO NO.              float64
CJ_Description          object
DC_Location             object
PO Pending             float64
PO_Status               object
Next Delivery          float64
Supplier_Short_Name     object
dtype: object


C:\Users\Thanawit C\AppData\Local\Temp\ipykernel_3060\2140401721.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df2.rename(columns={'Unit_PC/CAR': 'PC_Cartons'}, inplace=True)
C:\Users\Thanawit C\AppData\Local\Temp\ipykernel_3060\2140401721.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df2['SHM_Article'] = selected_df2['SHM_Article'].astype(str)


Checking data type of each column: CJ_Item                         object
SHM_Item                        object
PO_Qty_to_DC1                  float64
PO_Qty_to_DC2                  float64
PO_Qty_to_DC4                  float64
PO_Qty_to_TD09                 float64
Min_del_date_to_DC1     datetime64[ns]
Min_del_date_to_DC2     datetime64[ns]
Min_del_date_to_DC4     datetime64[ns]
Min_del_date_to_TD09    datetime64[ns]
dtype: object
PO pending HBA report has been saved to D:\Data for Stock Report\cleaned_PO_pending_HBA.xlsx
